In [1]:
# Load the Appropriate Libraries to Do the work
import pandas as pd
import numpy as np


In [2]:
# Load the NFL3D play_level csv file.

df = pd.read_csv("./plays_with_passer_expanded_with_z_norm.csv")
df.head(10) 


# Create the frames distribution

frames_prob = df.groupby('frames_to_pass').size().div(len(df))
frames_model = frames_prob.to_dict()


# Create the field spread distribution
T_model = {}

xs = np.linspace(0.5, 119.5, 120)
ys = np.linspace(-0.5, 53.5, 55)
ys[0] = -0.2 

for x in xs:
    for y in ys:
        pos = (x,y)
        T_model[pos] = []

for i,r in df.iterrows():
    for x_relax in (-4, 4, 1):
        for y_relax in (-4, 4, 1):
            try:
                x = int(r['end_x_norm'])+.5+x_relax-20
                y = int(r['end_y_norm'])+.5+y_relax
                T_model[(x,y)].append(r['air_time'])
            except:
                pass

print(T_model)

 36.5): [], (100.5, 37.5): [], (100.5, 38.5): [], (100.5, 39.5): [], (100.5, 40.5): [], (100.5, 41.5): [], (100.5, 42.5): [], (100.5, 43.5): [], (100.5, 44.5): [], (100.5, 45.5): [], (100.5, 46.5): [], (100.5, 47.5): [], (100.5, 48.5): [], (100.5, 49.5): [], (100.5, 50.5): [], (100.5, 51.5): [], (100.5, 52.5): [], (100.5, 53.5): [], (101.5, -0.2): [], (101.5, 0.5): [], (101.5, 1.5): [], (101.5, 2.5): [], (101.5, 3.5): [], (101.5, 4.5): [], (101.5, 5.5): [], (101.5, 6.5): [], (101.5, 7.5): [], (101.5, 8.5): [], (101.5, 9.5): [], (101.5, 10.5): [], (101.5, 11.5): [], (101.5, 12.5): [], (101.5, 13.5): [], (101.5, 14.5): [], (101.5, 15.5): [], (101.5, 16.5): [], (101.5, 17.5): [], (101.5, 18.5): [], (101.5, 19.5): [], (101.5, 20.5): [], (101.5, 21.5): [], (101.5, 22.5): [], (101.5, 23.5): [], (101.5, 24.5): [], (101.5, 25.5): [], (101.5, 26.5): [], (101.5, 27.5): [], (101.5, 28.5): [], (101.5, 29.5): [], (101.5, 30.5): [], (101.5, 31.5): [], (101.5, 32.5): [], (101.5, 33.5): [], (101.5, 34

First we will calculate the distribtition of the frames of throw this is $P(t)$. 

In [3]:
def probOfThrowOnFrame(frames_model, frames_since_snap):
    return frames_model[frames_since_snap]
    
probOfThrowOnFrame(frames_model, 10)
probOfThrowOnFrame(frames_model, 60)





0.0014989046466044045

In [4]:
x = np.linspace(0.5, 119.5, 120)
y = np.linspace(-0.5, 53.5, 55)
y[0] = -0.2  # football field width is 53.3
field_locs = np.stack(np.meshgrid(x, y)).reshape(2, -1).T  # (F, 2)
print(field_locs.shape)
def f(loc):
    x = loc[0]
    y = loc[1]
    v = (x,y)
    vals = T_model[v]
    if(len(vals) == 0): 
        temp = np.zeros(40)
        temp[-1,] = 1
        return temp
    N = 8
    b,bins = np.histogram(vals, bins = np.linspace(0,4, 48),density = True)
    if np.isnan(b).any():
        b = np.zeros_like(b, dtype=float)
        b[-1] = 1.
    b = np.convolve(b, np.ones(N)/N, mode='valid')
    normed = b.sum()
    b = b/normed
    return b
mapped = np.apply_along_axis(f, 1, field_locs)
#print(mapped[4000])
#print(mapped)

reshaped_mapped = mapped.reshape(55,120,40)
the_important_stuff = reshaped_mapped[:,10:80]
print(the_important_stuff.shape)
print(the_important_stuff[30][40])



(6600, 2)
(55, 70, 40)
[0.         0.         0.         0.         0.         0.
 0.         0.01388889 0.02777778 0.02777778 0.05555556 0.08333333
 0.09722222 0.09722222 0.11111111 0.09722222 0.08333333 0.08333333
 0.05555556 0.02777778 0.01388889 0.01388889 0.01388889 0.01388889
 0.01388889 0.01388889 0.01388889 0.01388889 0.01388889 0.01388889
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [5]:
t = np.linspace(0.1,4,40)
qb_x = 50
field_probs = the_important_stuff.astype(float)

maxT = np.zeros(len(t), dtype=float); maxT[-1] = 1


probs = np.column_stack((np.zeros((55, qb_x, len(t))),field_probs,np.tile(maxT, (55, 120-(70+qb_x), 1))))
print(probs.shape)

tots = []
for qbx in range(0,100):
    if(qbx <= 50):
        val = np.column_stack((np.zeros((55, qbx, len(t))),field_probs,np.tile(maxT, (55, 120-(70+qbx), 1))))
    else:
        val = np.column_stack((np.zeros((55, qbx, len(t))),field_probs))
        val = val[:,0:120,]
    val = val.reshape(-1,len(t))
    #print(val.shape)
    tots.append(val)
    
tots = np.array(tots)
print(tots.shape)



(55, 120, 40)
(100, 6600, 40)


In [6]:
np.save("../data/T_given_Ls.npy", tots)
